
# 🚗 AWS DynamoDB – Car Rentals Lab (Using boto3.client)

This lab demonstrates how to use **`boto3.client()`** to work with **DynamoDB** in a real-world **Car Rentals** use case.

We will create a table, insert records, query data, update items, and finally delete the table — all using **boto3.client()**.



## ✅ Prerequisites

- AWS account and credentials configured (`aws configure`)
- IAM user or role with `AmazonDynamoDBFullAccess`
- Python 3.7+ and `boto3` installed

```bash
pip install boto3
```



## ⚙️ Step 1 – Create DynamoDB Table

We’ll create a table named **CarRentals** with:
- **Partition key:** `RentalID` (String)
- **Sort key:** `CustomerID` (String)


In [ ]:

import boto3
import time

# Initialize DynamoDB client
dynamodb = boto3.client('dynamodb', region_name='us-east-1')

table_name = 'CarRentals'

# Create table
response = dynamodb.create_table(
    TableName=table_name,
    KeySchema=[
        {'AttributeName': 'RentalID', 'KeyType': 'HASH'},  # Partition key
        {'AttributeName': 'CustomerID', 'KeyType': 'RANGE'}  # Sort key
    ],
    AttributeDefinitions=[
        {'AttributeName': 'RentalID', 'AttributeType': 'S'},
        {'AttributeName': 'CustomerID', 'AttributeType': 'S'}
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print("Creating table...")
# Wait until the table is active
waiter = dynamodb.get_waiter('table_exists')
waiter.wait(TableName=table_name)
print(f"✅ Table '{table_name}' is active now.")



## 🚘 Step 2 – Insert Car Rental Records

Let’s insert diverse rental records showing DynamoDB’s schema flexibility.


In [ ]:

# Insert multiple car rental records
items = [
    {
        'RentalID': {'S': 'RNT1001'},
        'CustomerID': {'S': 'CUST001'},
        'CarModel': {'S': 'Toyota Camry'},
        'RentalDays': {'N': '3'},
        'PickupCity': {'S': 'Bangalore'},
        'DropoffCity': {'S': 'Mysuru'},
        'PaymentMode': {'S': 'CreditCard'},
        'TotalAmount': {'N': '7200'}
    },
    {
        'RentalID': {'S': 'RNT1002'},
        'CustomerID': {'S': 'CUST002'},
        'CarModel': {'S': 'Hyundai i20'},
        'RentalDays': {'N': '2'},
        'PickupCity': {'S': 'Chennai'},
        'Insurance': {'M': {'Type': {'S': 'Comprehensive'}, 'Provider': {'S': 'ICICI Lombard'}}},
        'Discount': {'N': '10'}
    },
    {
        'RentalID': {'S': 'RNT1003'},
        'CustomerID': {'S': 'CUST003'},
        'CarModel': {'S': 'Honda City'},
        'RentalDays': {'N': '4'},
        'Extras': {'L': [{'S': 'GPS'}, {'S': 'ChildSeat'}]},
        'PickupCity': {'S': 'Hyderabad'},
        'DropoffCity': {'S': 'Hyderabad'},
        'TotalAmount': {'N': '9000'}
    }
]

for item in items:
    dynamodb.put_item(TableName=table_name, Item=item)

print("✅ Inserted 3 car rental records successfully.")



## 🔍 Step 3 – Scan and Query the Table

We can scan all records or query specific rentals.


In [ ]:

# Scan all items in the table
response = dynamodb.scan(TableName=table_name)
print("All Rental Records:")
for item in response['Items']:
    print(item)


In [ ]:

# Query by RentalID
from boto3.dynamodb.conditions import Key

# DynamoDB client queries require KeyConditionExpression strings
response = dynamodb.query(
    TableName=table_name,
    KeyConditionExpression='RentalID = :r_id',
    ExpressionAttributeValues={':r_id': {'S': 'RNT1001'}}
)

print("Query result for RentalID = RNT1001:")
for item in response['Items']:
    print(item)



## ✏️ Step 4 – Update an Item
Update the `TotalAmount` for a specific rental.


In [ ]:

response = dynamodb.update_item(
    TableName=table_name,
    Key={
        'RentalID': {'S': 'RNT1001'},
        'CustomerID': {'S': 'CUST001'}
    },
    UpdateExpression='SET TotalAmount = :t',
    ExpressionAttributeValues={':t': {'N': '7500'}},
    ReturnValues='UPDATED_NEW'
)

print("✅ Updated TotalAmount:", response['Attributes'])



## ❌ Step 5 – Delete an Item
Remove one rental record from the table.


In [ ]:

response = dynamodb.delete_item(
    TableName=table_name,
    Key={
        'RentalID': {'S': 'RNT1002'},
        'CustomerID': {'S': 'CUST002'}
    }
)

print("✅ Deleted record RNT1002 successfully.")



## 🧹 Step 6 – Cleanup
Delete the DynamoDB table.


In [ ]:

dynamodb.delete_table(TableName=table_name)
print(f"🗑️ Deleting table '{table_name}'...")

# Wait until deletion completes
while True:
    try:
        dynamodb.describe_table(TableName=table_name)
        print("⏳ Waiting for table deletion...")
        time.sleep(5)
    except dynamodb.exceptions.ResourceNotFoundException:
        print(f"✅ Table '{table_name}' deleted successfully.")
        break
